In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.test.is_gpu_available():
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    

def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

def plot_color_image(image):
    plt.imshow(image, interpolation="nearest")
    plt.axis("off")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
No GPU was detected. CNNs can be very slow without a GPU.


In [50]:
default_reber_grammar = [
    [("B", 1)],
    [("T", 2), ("P", 3)],
    [("S", 2), ("X", 4)],
    [("T", 3), ("V", 5)],
    [("S", 6), ("X", 3)],
    [("P", 4), ("V", 6)],
    [("E", None)]
]

embedded_reber_grammar = [
    [("B", 1)],
    [("T", 2), ("P", 3)],
    [(default_reber_grammar, 4)],
    [(default_reber_grammar, 5)],
    [("T", 6)],
    [("P", 6)],
    [("E", None)],
]

def reber_grammar_generator(grammar):
    result = []

    curr_state_idx = 0
    while curr_state_idx is not None:
        path_id = np.random.randint(len(grammar[curr_state_idx]))
        node, new_state_idx = grammar[curr_state_idx][path_id]
        if(isinstance(node, list)):
            node = reber_grammar_generator(node)
        result.append(node)
        curr_state_idx = new_state_idx
    
    return "".join(result)

POSSIBLE_CHARS = "BEPSTVX"
def bad_grammar_generator(grammar):
#     POSSIBLE_CHARS = "12345678"
    good_grammar = reber_grammar_generator(grammar)
    rand_pos = np.random.randint(len(good_grammar))
    good_char = good_grammar[rand_pos]
    bad_char = np.random.choice(sorted(set(POSSIBLE_CHARS) - set(good_char)))
#     print(good_grammar)
#     print("->".join([good_char, bad_char]))
    
    return good_grammar[:rand_pos] + bad_char + good_grammar[rand_pos + 1:]

def encode_string(string):
    return [POSSIBLE_CHARS.index(_) + 1 for _ in string]

In [55]:
def get_N_samples_of_good_grammar(num, grammar):
    return [encode_string(reber_grammar_generator(grammar)) for _ in range(num)], [1. for _ in range(num)]
def get_N_samples_of_bad_grammar(num, grammar):
    return [encode_string(bad_grammar_generator(grammar)) for _ in range(num)], [0. for _ in range(num)]

def get_dataset(num):
    x_good_grammar, y_good_grammar = get_N_samples_of_good_grammar(num // 2, default_reber_grammar)
    x_bad_grammar, y_bad_grammar = get_N_samples_of_bad_grammar(num // 2, default_reber_grammar)

    __X = tf.ragged.constant(x_good_grammar + x_bad_grammar)
    __y = tf.constant(y_good_grammar + y_bad_grammar)

    return tf.data.Dataset.from_tensor_slices((__X, __y)).shuffle(10000).batch(16)



In [56]:
train_ds = get_dataset(10000)
valid_ds = get_dataset(2000)


In [44]:
def get_dataset_2(num):
    x_good_grammar, y_good_grammar = get_N_samples_of_good_grammar(num // 2, default_reber_grammar)
    x_bad_grammar, y_bad_grammar = get_N_samples_of_bad_grammar(num // 2, default_reber_grammar)

    __X = tf.ragged.constant(x_good_grammar + x_bad_grammar, ragged_rank=1)
    __y = np.array(y_good_grammar + y_bad_grammar)

    return __X, __y
train_ds_x1, train_ds_y1 = get_dataset_2(10000)


In [62]:

embedding_size = 5

model = keras.models.Sequential([
#     keras.layers.InputLayer(input_shape=[None], dtype=tf.int32, ragged=True),
    keras.layers.Embedding(input_dim=len(POSSIBLE_CHARS)+1, output_dim=embedding_size),
    keras.layers.GRU(30),
    keras.layers.Dense(1, activation="sigmoid")
])
# optimizer = keras.optimizers.SGD(lr=0.02, momentum = 0.95, nesterov=True)
# model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
# history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

In [63]:
optimizer = keras.optimizers.SGD(lr=0.02, momentum = 0.95, nesterov=True)
model.compile(loss = "binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(train_ds, epochs=4, validation_data=valid_ds)
# history = model.fit(train_ds_x1, train_ds_y1, epochs=5)

Epoch 1/4
625/625 [==============================] - 5s 8ms/step - loss: 0.6764 - accuracy: 0.5415 - val_loss: 0.6322 - val_accuracy: 0.6300
Epoch 2/4
625/625 [==============================] - 5s 7ms/step - loss: 0.5970 - accuracy: 0.6567 - val_loss: 0.5150 - val_accuracy: 0.7620
Epoch 3/4
625/625 [==============================] - 5s 7ms/step - loss: 0.4163 - accuracy: 0.8121 - val_loss: 0.2631 - val_accuracy: 0.9065
Epoch 4/4
625/625 [==============================] - 5s 8ms/step - loss: 0.1663 - accuracy: 0.9409 - val_loss: 0.0803 - val_accuracy: 0.9765


In [59]:
for __X, __y in train_ds.take(1):
    for ___X, ___y in zip(__X, __y):
#         print(___X.numpy(), ___y.numpy())
        for idx in ___X.numpy():
            if(idx > 0):
                print(POSSIBLE_CHARS[idx-1], end="")
        print("->", ___y.numpy())

BTXXVPXVVE-> 1.0
BPTVPSE-> 1.0
BTSXXVPSE-> 1.0
BPTVVE-> 1.0
BTPSE-> 0.0
BPTTTTVSSE-> 0.0
BETVPXVVE-> 0.0
BSTVPSE-> 0.0
BPSXXVVE-> 0.0
BXXSE-> 0.0
BPTVPXTVVE-> 1.0
BPVVE-> 1.0
VTXXVVE-> 0.0
BTSSSXSE-> 1.0
BTXTE-> 0.0
BTXXVPSE-> 1.0
